In [1]:
from osgeo import gdal
from osgeo import ogr
import os, sys
import numpy as np
import json

In [2]:
def extract_coord(output_poly): #fonction pour extraire les coordonnées géographiques à partir d'un fichier json
    
    extract_json = json.loads(output_poly)
    coordinates = extract_json['coordinates']
    xmin = coordinates[0][0][0][0]
    xmax = coordinates[0][0][1][0]
    ymin = coordinates[0][0][0][1]
    ymax = coordinates[0][0][2][1]
    
    return xmin,ymin,xmax,ymax

In [22]:
def merge_RGBNIR(listRGBNIR,output_dir):
    gdal.BuildVRT("Images/RGBNIR.vrt",listRGBNIR, separate=True)
    output = output_dir+'merge_RGBNIR.tif'
    gdal.Translate(output_dir+merge_RGBNIR+'.tif',"Images/RGBNIR.vrt", format='GTiff')
    
merge_RGBNIR(["Images/S2/S2_B04_2019_Q1_small.tif","Images/S2/S2_B03_2019_Q1_small.tif","Images/S2/S2_B02_2019_Q1_small.tif","Images/S2/S2_B08_2019_Q1_small.tif"],'Images/S2/')
    

In [56]:
#code from https://github.com/tgrippa/Partimap/blob/cd72ecd2f3e32794eb3e0ee70399de1811b5399e/Notebooks/Data_preprocessing.ipynb
def normalise_01(image_data):
    image_data -= np.min(stack, axis=0)
    image_data /= (np.max(stack, axis=0) - np.min(stack, axis=0))
    return image_data

In [26]:
def chunckage(folder_raster,filename_raster,format_rast,folder_vector, filename_vector):
    
    rast = gdal.Open(folder_raster+filename_raster+format_rast) #ouverture du raster

    vector = ogr.Open(folder_vector+filename_vector) #ouverture de la fichier gkpg

    layer = vector.GetLayer() #extraction des couches dans le gkpg dans ce cas-ci il n'y en a qu'une seule (sinon spécifier)

    print(layer.GetExtent()) #affiche de l'emprise de la couche vectorielle

    listvrt = []
    i=0
    
    #os.system('cd '+folder_raster)
    #os.system('sudo mkdir /patch')

    for feat in layer: #une boucle sur les polygones dans la couche vectorielle
        if i <= 10 : 
            polygone = feat.GetGeometryRef().ExportToJson() #extraire les données géométriques et exporter en format JSON
            list_coord = extract_coord(polygone) #extraction des coordonnées des polygones
            print(list_coord)
            poly_id = feat.GetField("id") #extraire le id de l'object
            lulc = feat.GetField("lulc") #extraire le lulc (si slum ou pas slum) de l'object
            name_file= filename_raster+'_'+str(poly_id)+'_'+str(lulc)+'.tif' #création du nom du fichier de sortie
            filename = folder_raster+filename_raster+'_patch/'+str(name_file)
            gdal.Warp(str(filename),rast, outputBounds = list_coord) #division en petite tuile correspondante aux polygone dans le shpfile
            listvrt.append(filename)
            i +=1
            
    return listvrt
              
merge_rbgnir_list = chunckage("Images/S2/","merge_RGBNIR",".tif",'Images/smaller/sample/','sample_grid.gpkg')


(259690.0, 266590.0, 9853680.0, 9860280.0)
(259690.0, 9860280.0, 259790.0, 9860180.0)
(259690.0, 9860180.0, 259790.0, 9860080.0)
(259690.0, 9860080.0, 259790.0, 9859980.0)
(259690.0, 9859980.0, 259790.0, 9859880.0)
(259690.0, 9859880.0, 259790.0, 9859780.0)
(259690.0, 9859780.0, 259790.0, 9859680.0)
(259690.0, 9859680.0, 259790.0, 9859580.0)
(259690.0, 9859580.0, 259790.0, 9859480.0)
(259690.0, 9859480.0, 259790.0, 9859380.0)
(259690.0, 9859380.0, 259790.0, 9859280.0)
(259690.0, 9859280.0, 259790.0, 9859180.0)


In [60]:
#code inspired from https://github.com/tgrippa/Partimap/blob/cd72ecd2f3e32794eb3e0ee70399de1811b5399e/Notebooks/Data_preprocessing.ipynb
stack = []
for patch in merge_rbgnir_list :
    pat = gdal.Open(patch)
    data_rast = pat.ReadAsArray().astype(np.float)
    data_rast = np.transpose(data_rast,(1,2,0))
    stack.append(data_rast)

stack= np.array(stack)
print(stack)
print(stack.shape)
print(stack.dtype)

stack = normalise_01(stack)



[[[[1136.  749.  490. 1743.]
   [1408. 1034.  720. 2520.]
   [1272. 1110.  801. 2480.]
   ...
   [1488. 1418. 1272. 1992.]
   [1090. 1042.  817. 2546.]
   [ 710.  692.  433. 2802.]]

  [[1134.  973.  666. 2604.]
   [1196. 1064.  772. 2878.]
   [ 985.  940.  661. 2778.]
   ...
   [1434. 1318. 1150. 2400.]
   [1034.  875.  599. 2394.]
   [ 869.  858.  576. 2540.]]

  [[ 641.  672.  405. 2731.]
   [ 561.  601.  351. 2698.]
   [ 597.  608.  333. 2677.]
   ...
   [1304. 1011.  810. 2084.]
   [1152.  978.  684. 2472.]
   [1332. 1074.  725. 2574.]]

  ...

  [[ 970.  805.  501. 2364.]
   [ 953.  788.  502. 2376.]
   [ 878.  760.  471. 2314.]
   ...
   [1100.  877.  668. 1772.]
   [ 761.  699.  465. 1266.]
   [ 971.  833.  720. 1110.]]

  [[1052.  886.  539. 2359.]
   [ 999.  824.  490. 2385.]
   [ 830.  767.  485. 2216.]
   ...
   [ 987.  785.  555. 1665.]
   [ 733.  686.  523. 1183.]
   [ 834.  684.  520. 1038.]]

  [[1358. 1062.  694. 2311.]
   [1216.  958.  623. 2310.]
   [ 862.  785.  496

<ipython-input-60-865bae75f21f>:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data_rast = pat.ReadAsArray().astype(np.float)


In [61]:
print(stack)

[[[[0.08446866 0.         0.         0.49749373]
   [0.55901427 0.2863706  0.         1.        ]
   [0.30892742 0.32128514 0.         0.85961538]
   ...
   [0.62637363 0.90749601 1.         0.67148014]
   [0.         0.06918239 0.05736138 1.        ]
   [0.         0.         0.         1.        ]]

  [[0.43441637 0.54765507 0.27521368 1.        ]
   [0.32237674 0.42091837 0.19891008 1.        ]
   [0.05996622 0.16848485 0.         1.        ]
   ...
   [0.74233129 1.         1.         1.        ]
   [0.         0.         0.         1.        ]
   [0.         0.02879581 0.00292683 1.        ]]

  [[0.         0.05507246 0.         1.        ]
   [0.         0.02971429 0.         1.        ]
   [0.         0.         0.         1.        ]
   ...
   [0.45617978 0.36979969 0.38223938 0.78195489]
   [0.         0.1745283  0.         1.        ]
   [0.3814433  0.38910506 0.14663951 1.        ]]

  ...

  [[0.03407756 0.         0.         1.        ]
   [0.         0.         0.       